<a href="https://colab.research.google.com/github/ImbangJayoGampito/DIF62130_B_25_2311532002/blob/main/Praktikum%205/GaussianNB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix,classification_report, roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
dataset_dict = {
    'Rainfall': [0.0, 2.0, 7.0, 18.0, 3.0, 3.0, 0.0, 1.0, 0.0, 25.0, 0.0, 18.0, 9.0, 5.0, 0.0, 1.0, 7.0, 0.0, 0.0, 7.0, 5.0, 3.0, 0.0, 2.0, 0.0, 8.0, 4.0, 4.0],
    'Temperature': [29.4, 26.7, 28.3, 21.1, 20.0, 18.3, 17.8, 22.2, 20.6, 23.9, 23.9, 22.2, 27.2, 21.7, 27.2, 23.3, 24.4, 25.6, 27.8, 19.4, 29.4, 22.8, 31.1, 25.0, 26.1, 26.7, 18.9, 28.9],
    'Humidity': [85.0, 90.0, 78.0, 96.0, 80.0, 70.0, 65.0, 95.0, 70.0, 80.0, 70.0, 90.0, 75.0, 80.0, 88.0, 92.0, 85.0, 75.0, 92.0, 90.0, 85.0, 88.0, 65.0, 70.0, 60.0, 95.0, 70.0, 78.0],
    'WindSpeed': [2.1, 21.2, 1.5, 3.3, 2.0, 17.4, 14.9, 6.9, 2.7, 1.6, 30.3, 10.9, 3.0, 7.5, 10.3, 3.0, 3.9, 21.9, 2.6, 17.3, 9.6, 1.9, 16.0, 4.6, 3.2, 8.3, 3.2, 2.2],
    'Play': ['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'Yes', 'Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes']
}
df = pd.DataFrame(dataset_dict)
df.head()

,Rainfall,Temperature,Humidity,WindSpeed,Play
0,0.0,29.4,85.0,2.1,No
1,2.0,26.7,90.0,21.2,No
2,7.0,28.3,78.0,1.5,Yes
3,18.0,21.1,96.0,3.3,Yes
4,3.0,20.0,80.0,2.0,Yes


In [4]:
df['Play'] = (df['Play'] == "Yes").astype(int)
df.head()

,Rainfall,Temperature,Humidity,WindSpeed,Play
0,0.0,29.4,85.0,2.1,0
1,2.0,26.7,90.0,21.2,0
2,7.0,28.3,78.0,1.5,1
3,18.0,21.1,96.0,3.3,1
4,3.0,20.0,80.0,2.0,1


In [17]:
x = df.drop('Play', axis=1)
y = df['Play']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
x_test.head()

,Rainfall,Temperature,Humidity,WindSpeed
9,25.0,23.9,80.0,1.6
25,8.0,26.7,95.0,8.3
8,0.0,20.6,70.0,2.7
21,3.0,22.8,88.0,1.9
0,0.0,29.4,85.0,2.1


In [26]:
from fractions import Fraction
def gaussian_nb(x_train, y_train, x_test , y_test):
  total_count = y_train.value_counts().sum()
  prob_series = y_train.value_counts().apply(lambda x: Fraction(x,total_count))
  prior = prob_series
  stats = {}
  for cls in range (0, len(prob_series)):
    x_train_cls = x_train[y_train == cls]
    stats[cls] = {}
    for col in x_train_cls.columns:
      stats[cls][col] = {}
      stats[cls][col] = {
                'mean': x_train_cls[col].mean(),
                'std': x_train_cls[col].std() if x_train_cls[col].std() > 0 else 1e-6
            }
  res = {}
  for index, row in x_test.iterrows():
    predicted_class = index
    highest_prob = 0
    for cls in prior.index:
      prob = prior[cls]
      for col in row.index:
        mean = stats[cls][col]['mean']
        std = stats[cls][col]['std']
        prob *= (1 / (np.sqrt(2 * np.pi) * std)) * np.exp(-((row[col] - mean) ** 2) / (2 * std ** 2))
      if prob > highest_prob:
        highest_prob = prob
        predicted_class = cls
    res[index] = predicted_class
  return pd.Series(res)
y_pred = gaussian_nb(x_train, y_train, x_test , y_test)

In [27]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7777777777777778
